In [21]:
from fractions import Fraction
from collections import defaultdict
import numpy as np
import random
from itertools import combinations, product

# GENERATE THE HYPERCUBE-----------------------------------------------------------

def generate_combinations(points, k):
    """
    Generates all possible subsets of size k.
    """
    subsets = list(combinations(points, k))
    return subsets

# For the hypercube:
def generate_vertices(d):
    """
    Generate all vertices of the hypercube.
    """
    return list(product([-1, 1], repeat=d))

def generate_edges(vertices):
    """
    Generate all edges of the hypercube.
    """
    edges = []
    num_vertices = len(vertices)
    
    # Check all pairs of vertices
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            # Count the number of differing coordinates
            differences = sum(1 for a, b in zip(vertices[i], vertices[j]) if a != b)
            if differences == 1:
                edges.append((vertices[i], vertices[j]))
    return edges

def d_cube(d):
    """
    Generate the vertices and edges of a d-dimensional hypercube.
    """
    return generate_vertices(d), generate_edges(generate_vertices(d))

# UTILITY FUNCTIONS-----------------------------------------------------------

def dot_product_fraction(point, reference):
    """
    Computes the dot product between a point and a reference vector using fractions.
    """
    return sum(Fraction(x) * y for x, y in zip(point, reference))

def group_by_first_entry(dot_products):
    """
    Groups points by their first entry (dot product value) into a list.
    """
    groups = defaultdict(list)
    for product, point in dot_products:
        groups[product].append(point)
    return list(groups.values())

def sort_points_by_dot_product(points, reference):
    """
    Sorts points based on their dot product with a reference vector using fractions.
    """
    reference_frac = [Fraction(x) for x in reference]
    dot_products = [(dot_product_fraction(point, reference_frac), point) for point in points]
    dot_products.sort(key=lambda x: x[0])
    return group_by_first_entry(dot_products)

def compute_slices(order, edges):
    """
    Computes the slices by vertices and edges and generates an interleaved list combining both results.
    """
    vertex_slices = []  # List for slices by vertices
    edge_slices = []    # List for slices by edges
    
    # Compute slices by vertices
    for i in range(1, len(order) - 1): 
        subset1 = [x for group in order[:i] for x in group]
        subset2 = [x for group in order[i + 1:] for x in group]
        count = len(order[i])
        for e in edges:
            if (e[0] in subset1 and e[1] in subset2) or (e[1] in subset1 and e[0] in subset2):
                count += 1
        vertex_slices.append(count)

    # Compute slices by edges
    for i in range(1, len(order)):
        subset1 = [v for sub in order[:i] for v in sub]
        subset2 = [v for sub in order[i:] for v in sub]
        count = 0
        for e in edges:
            if (e[0] in subset1 and e[1] in subset2) or (e[1] in subset1 and e[0] in subset2):
                count += 1
        edge_slices.append(count)

    # Verify compatibility of list lengths
    if len(edge_slices) != len(vertex_slices) + 1:
        raise ValueError("The length of edge_slices must be exactly one more than that of vertex_slices")

    # Interleave results
    result = []
    for i in range(len(vertex_slices)):
        result.append(edge_slices[i])
        result.append(vertex_slices[i])
    result.append(edge_slices[-1])

    return result

# MAIN------------------------------------------------------

d = 6

# Generate hypercube vertices and edges
Q_d = d_cube(d)
V_Q = Q_d[0]
E_Q = Q_d[1]
        
directions = [
    [1.0, 1.0, -0.9, -0.9, -0.5, -0.5],
    [1.0, 1.0, -0.9, -0.9, -0.5, 0.0],
    [1.0, -0.5, 1.0, 1.0, 1.0, -0.5],
    [1.0, -0.5, 1.0, -0.5, -0.9, 1.0],
    [1.0, -0.5, 1.0, -0.5, -0.9, -0.5],
    [1.0, -0.5, -0.5, -0.5, -0.5, -0.9],
    [1.0, -0.5, -0.9, 1.0, -0.9, -0.9],
    [1.0, -0.9, 1.0, 1.0, 1.0, -0.5],
    [1.0, -0.9, 1.0, 1.0, -0.5, -0.9],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    [1.0, -0.5, -0.5, -0.5, -0.9, -0.9],
    [Fraction(3, 500), Fraction(3, 1000), Fraction(3, 1000), Fraction(7, 1000), Fraction(3, 500), Fraction(3, 1000)],
    [Fraction(3, 1000),Fraction(9, 1000), Fraction(3, 500), Fraction(3, 500), Fraction(7, 1000), Fraction(3, 1000)],
    [Fraction(3, 500), Fraction(1, 100), Fraction(7, 1000), Fraction(3, 500), Fraction(9, 1000), Fraction(3, 1000)],
    [Fraction(9, 1000),Fraction(1, 200), Fraction(7, 1000), Fraction(1, 250), Fraction(1, 500), Fraction(7, 1000)]
]

    
    
# Ensure cortes_direccion is computed
cortes_direccion = []
for direction in directions:
    order = sort_points_by_dot_product(V_Q, direction)
    slices = compute_slices(order, E_Q)
    cortes_direccion.append(slices)

# Initialize lists to store directions and unique slices
directions_and_slices = [[], []]
accumulated_slices = set()

# Iterate through all directions
for i in range(len(directions)):
    if i >= len(cortes_direccion):
        print(f"Skipping direction {i}: No corresponding slice data.")
        continue
    
    # Add the current direction to the first list
    directions_and_slices[0].append(directions[i])
    
    # Sort and remove duplicates of slices in the current direction
    sorted_slices = sorted(set(cortes_direccion[i]))
    directions_and_slices[1].append(sorted_slices)
    
    # Update the accumulated set of unique slices
    accumulated_slices |= set(sorted_slices)
    
    # Print the accumulated slices up to this point
    print('Accumulated slices:', accumulated_slices)



Accumulated slices: {6, 14, 21, 22, 28, 32, 36, 40, 42, 44, 47, 48, 50, 53, 56, 58, 60}
Accumulated slices: {6, 10, 14, 16, 21, 22, 28, 32, 36, 40, 42, 44, 46, 47, 48, 50, 53, 56, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 21, 22, 24, 28, 32, 34, 36, 40, 42, 44, 46, 47, 48, 50, 53, 56, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 17, 18, 21, 22, 24, 28, 30, 32, 34, 36, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 56, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 17, 18, 19, 21, 22, 24, 28, 30, 32, 33, 34, 36, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 56, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 26, 28, 30, 32, 33, 34, 36, 38, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 56, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 30, 32, 33, 34, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 56, 57, 58, 60}
Accumulated slices: {6, 10, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 30, 32, 33, 